# CONTENT BASED FILTERING

## Import Library

In [42]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

In [2]:
file_path = r'C:\Users\user\Documents\ZENROOM\dataset_ruangan.csv'
df = pd.read_csv(file_path)

In [14]:
print("Initial Dataset:")
print(df.head())

Initial Dataset:
    ID Kode                         Nama  Lantai  \
0  739  NaN  Ruang Scienfic Lecture Hall       1   
1  736  103                          NaN       1   
2  227  303                         AULA       3   
3   83  102                          NaN       1   
4  193  NaN           Ruang Sidang Senat       3   

                                Fakultas       Status  Kapasitas     Luas  \
0                                    TPB  Tidak Aktif        500   750.00   
1                                    TPB  Tidak Aktif        210     0.00   
2  Fakultas Keguruan dan Ilmu Pendidikan        Aktif        200   218.90   
3                        Fakultas Teknik  Tidak Aktif        200  1108.42   
4  Fakultas Keguruan dan Ilmu Pendidikan        Aktif        200   296.00   

   Rating   Tujuan  
0     4.2   Sidang  
1     4.3    Kelas  
2     3.9  Seminar  
3     3.5   Sidang  
4     3.9  Seminar  


In [15]:
# Hapus ruangan dengan status 'Tidak Aktif'
cleaned_data = df[df['Status'] == 'Aktif']

# Preview cleaned data
print("\nDataset after removing 'Tidak Aktif' rooms:")
print(cleaned_data.head())


Dataset after removing 'Tidak Aktif' rooms:
    ID     Kode                Nama  Lantai  \
2  227      303                AULA       3   
4  193      NaN  Ruang Sidang Senat       3   
7  507     B3-E             Kelas E       3   
8  510     B2-D             Kelas D       2   
9  577  ATB.3.1             ATB.3.1       3   

                                Fakultas Status  Kapasitas   Luas  Rating  \
2  Fakultas Keguruan dan Ilmu Pendidikan  Aktif        200  218.9     3.9   
4  Fakultas Keguruan dan Ilmu Pendidikan  Aktif        200  296.0     3.9   
7                    Fakultas Kedokteran  Aktif        160  220.0     3.9   
8                    Fakultas Kedokteran  Aktif        160  220.0     3.6   
9                    Fakultas Peternakan  Aktif        150  150.0     3.5   

    Tujuan  
2  Seminar  
4  Seminar  
7    Kelas  
8  Seminar  
9  Seminar  


In [5]:
# Urutkan berdasarkan fakultas
df_sorted = cleaned_data.sort_values(by='Fakultas')

In [6]:
print(df_sorted)

      ID          Kode          Nama  Lantai          Fakultas Status  \
369  311       A.C.2.2  Ruang Kuliah       2  Fakultas Ekonomi  Aktif   
260  336  A.D.1.4 (D3)  Ruang Kuliah       1  Fakultas Ekonomi  Aktif   
265  328  A.D.3.2 (D3)  Ruang Kuliah       3  Fakultas Ekonomi  Aktif   
267  306       A.B.3.1  Ruang Kuliah       3  Fakultas Ekonomi  Aktif   
273  878      A.B.2.2B  Ruang Kuliah       2  Fakultas Ekonomi  Aktif   
..   ...           ...           ...     ...               ...    ...   
174  721           307       GKB-307       3               TPB  Aktif   
173  217           202       GKB-202       2               TPB  Aktif   
171  720           306       GKB-306       3               TPB  Aktif   
190  712           205       GKB-205       2               TPB  Aktif   
199  729           501       GKB-501       5               TPB  Aktif   

     Kapasitas   Luas  Rating   Tujuan  
369         40    NaN     4.1    Rapat  
260         50    NaN     3.6   Sidang  


In [7]:
df_sorted.replace('NA', pd.NA, inplace=True)

# Menghapus baris dengan nilai NaN
df_cleaned = df_sorted.dropna()

In [16]:
# Menghapus baris dengan nilai NaN
df_cleaned = df_sorted.dropna()

# Menampilkan DataFrame yang sudah dibersihkan
print("\nCleaned Dataset:")
print(df_cleaned.head())


Cleaned Dataset:
      ID          Kode                             Nama  Lantai  \
570  275  A.A.3.Lab. 2                    Lab. Komputer       3   
571  292       A.C.3.1                     Ruang Kuliah       3   
667  893       A.E.1.2  Ruang Kuliah, seminar dan Ujian       1   
894  347    A.D.1.Lab1                     Lab Komputer       1   
380   43    C1_R.MIH A                Ruang Kelas MIH A       1   

             Fakultas Status  Kapasitas   Luas  Rating   Tujuan  
570  Fakultas Ekonomi  Aktif         20   0.00     4.1    Rapat  
571  Fakultas Ekonomi  Aktif         20  66.75     4.4    Rapat  
667  Fakultas Ekonomi  Aktif         10   0.00     3.8  Seminar  
894  Fakultas Ekonomi  Aktif          0   0.00     3.5    Kelas  
380    Fakultas Hukum  Aktif         40  80.00     3.6   Sidang  


In [9]:
# Opsional: Menyimpan DataFrame yang sudah dibersihkan ke file CSV
df_cleaned.to_csv('cleaning_data.csv', index=False)

## Preprocessing-Data

In [26]:
# Membuat fitur gabungan dari beberapa kolom untuk content-based filtering
df_cleaned['combined_features'] = df_cleaned.apply(lambda row: f"{row['Lantai']} {row['Fakultas']} {row['Kapasitas']} {row['Rating']} {row['Tujuan']}", axis=1)

# Inisialisasi CountVectorizer untuk mengubah teks menjadi vektor fitur
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
count_matrix = count.fit_transform(df_cleaned['combined_features'])

# Menghitung matriks kemiripan kosinus berdasarkan vektor fitur
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# Simpan matriks kemiripan untuk penggunaan di masa depan
np.save('cosine_sim.npy', cosine_sim)

# Simpan DataFrame yang telah dibersihkan
df_cleaned.to_csv('final_cleaned_data.csv', index=False)

C:\Users\user\AppData\Local\Temp\ipykernel_17128\2333381807.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['combined_features'] = df_cleaned.apply(lambda row: f"{row['Lantai']} {row['Fakultas']} {row['Kapasitas']} {row['Rating']} {row['Tujuan']}", axis=1)


## Building models, Training, and Testing

*Pada tahap ini, saya berfokus pada penerapan model rekomendasi berbasis konten dan konversi ke TensorFlow Lite. Pertama, dilakukan pembacaan matriks similarity kosinus yang telah disimpan ('cosine_sim.npy') dan DataFrame akhir yang telah dibersihkan ('final_cleaned_data.csv'). Fungsi `get_content_based_recommendations` digunakan untuk memberikan rekomendasi ruangan berdasarkan similarity terhadap ruangan dengan ID tertentu. Jika ID ruangan tidak ditemukan dalam dataset, akan dihasilkan pesan kesalahan. Contoh penggunaan model diberikan dengan ID ruangan 347, yang menghasilkan rekomendasi berdasarkan similarity tertinggi dengan ruangan tersebut, dengan menampilkan atribut-atribut penting seperti ID, Nama, Lantai, Fakultas, Kapasitas, Luas, Rating, dan Tujuan.*

In [39]:
# Load cosine similarity matrix
cosine_sim = np.load(r'C:\Users\user\Documents\ZENROOM\cosine_sim.npy')

# Load final cleaned DataFrame
df_cleaned = pd.read_csv(r'C:\Users\user\Documents\ZENROOM\final_cleaned_data.csv')

# Contoh penggunaan model untuk mendapatkan rekomendasi
def get_content_based_recommendations(room_id, df_cleaned, cosine_sim, top_n=5):
    if room_id not in df_cleaned['ID'].values:
        raise ValueError(f"Room ID {room_id} tidak ditemukan dalam dataset.")
    idx = df_cleaned.index[df_cleaned['ID'] == room_id].tolist()[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    room_indices = [i[0] for i in sim_scores]
    return df_cleaned.iloc[room_indices]

# Contoh penggunaan model
room_id = 347
try:
    recommendations = get_content_based_recommendations(room_id, df_cleaned, cosine_sim)
    # Tampilkan hasil rekomendasi
    print("Content-Based Recommendations:")
    print(recommendations[['ID', 'Nama', 'Lantai', 'Fakultas', 'Kapasitas', 'Luas', 'Rating', 'Tujuan']])
except ValueError as e:
    print(e)

Content-Based Recommendations:
    ID                             Nama  Lantai          Fakultas  Kapasitas  \
0  275                    Lab. Komputer       3  Fakultas Ekonomi         20   
1  292                     Ruang Kuliah       3  Fakultas Ekonomi         20   
2  893  Ruang Kuliah, seminar dan Ujian       1  Fakultas Ekonomi         10   
5   26                 Ruang Kelas AH.7       2    Fakultas Hukum         80   
9   38                Ruang Kelas AH.18       3    Fakultas Hukum         80   

    Luas  Rating   Tujuan  
0   0.00     4.1    Rapat  
1  66.75     4.4    Rapat  
2   0.00     3.8  Seminar  
5  80.00     4.6    Kelas  
9  80.00     4.2    Kelas  


In [52]:
# Load cleaned dataframe and similarity matrix
df_cleaned = pd.read_csv('final_cleaned_data.csv')
cosine_sim = np.load('cosine_sim.npy')

# Create a tensor version of the dataframe
df_ids = df_cleaned['ID'].values.astype(np.int32)
df_ids_tensor = tf.constant(df_ids)

class ContentBasedModel(tf.Module):
    def __init__(self, sim_matrix, df_ids_tensor):
        super(ContentBasedModel, self).__init__()
        self.sim_matrix = tf.convert_to_tensor(sim_matrix, dtype=tf.float32)
        self.df_ids_tensor = tf.convert_to_tensor(df_ids_tensor, dtype=tf.int32)
    
    @tf.function(input_signature=[tf.TensorSpec(shape=(), dtype=tf.int32)])
    def recommend(self, room_id):
        room_idx = tf.where(self.df_ids_tensor == room_id)[0][0]
        sim_scores = self.sim_matrix[room_idx]
        top_indices = tf.argsort(sim_scores, direction='DESCENDING')[1:6]
        return top_indices

# Instantiate the model
model = ContentBasedModel(cosine_sim, df_ids_tensor)

# Save the model
tf.saved_model.save(model, "content_based_model_tf")

print("Model TensorFlow telah tersimpan di 'content_based_model_tf'.")

INFO:tensorflow:Assets written to: content_based_model_tf\assets
Model TensorFlow telah tersimpan di 'content_based_model_tf'.


In [53]:
# Convert to TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_saved_model("content_based_model_tf")
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open("content_based_model.tflite", "wb") as f:
    f.write(tflite_model)

print("Model has been converted to TensorFlow Lite and saved as 'content_based_model.tflite'.")

Model has been converted to TensorFlow Lite and saved as 'content_based_model.tflite'.


In [64]:
# Load cleaned dataframe
df_cleaned = pd.read_csv('final_cleaned_data.csv')

# Pastikan file model tflite ada di jalur yg benar
model_path = "content_based_model.tflite"

# Periksa apakah file model ada
if not os.path.exists(model_path):
    raise ValueError(f"Model file '{model_path}' tidak ditemukan. Pastikan file tersebut ada di direktori yang benar.")

# Load TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Input details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Masukkan ID ruangan yg ingin direkomendasikan
room_id = 514

# Konversi ID ke bentuk yang sesuai dengan input model
input_data = np.array([room_id], dtype=np.int32).reshape(input_details[0]['shape'])
interpreter.set_tensor(input_details[0]['index'], input_data)

# Jalankan inferensi
interpreter.invoke()

# Dapatkan output
output_data = interpreter.get_tensor(output_details[0]['index'])

# Tampilkan hasil rekomendasi
recommended_indices = output_data 
print("Content-Based Recommendations:")
for idx in recommended_indices:
    print(df_cleaned.iloc[idx][['ID', 'Nama', 'Lantai', 'Fakultas', 'Kapasitas', 'Luas', 'Rating', 'Tujuan']])


Content-Based Recommendations:
ID                           514
Nama           Lab. Mikrobiologi
Lantai                         1
Fakultas     Fakultas Kedokteran
Kapasitas                     25
Luas                        20.0
Rating                       3.3
Tujuan                   Seminar
Name: 84, dtype: object
ID                           534
Nama           OSCE CENTER 2- R1
Lantai                         2
Fakultas     Fakultas Kedokteran
Kapasitas                     12
Luas                        16.0
Rating                       3.8
Tujuan                   Seminar
Name: 36, dtype: object
ID                           532
Nama           OSCE CENTER 2- R3
Lantai                         2
Fakultas     Fakultas Kedokteran
Kapasitas                     12
Luas                        16.0
Rating                       3.0
Tujuan                   Seminar
Name: 44, dtype: object
ID                           531
Nama           OSCE CENTER 2- R4
Lantai                         2
Fakult

*Code tersebut bertujuan untuk melakukan integrasi model TensorFlow Lite yang telah disimpan (`content_based_model.tflite`) dengan aplikasi untuk memberikan rekomendasi berbasis konten. Pertama, kode memuat DataFrame yang telah dibersihkan dari file CSV (`final_cleaned_data.csv`). Kemudian, dilakukan pemeriksaan apakah file model TFLite (`content_based_model.tflite`) ada di jalur yang benar. Jika file tidak ditemukan, program akan menghasilkan error. Setelah memastikan keberadaan file model, TensorFlow Lite model diinisialisasi menggunakan `tf.lite.Interpreter` dan dilakukan alokasi tensor. Selanjutnya, kode menentukan detail input dan output dari model, lalu mempersiapkan data input berupa ID ruangan (`room_id`) yang ingin direkomendasikan. ID ini diubah menjadi format yang sesuai dengan input model dan di-set sebagai tensor input. Setelah itu, dilakukan inferensi dengan memanggil `interpreter.invoke()` untuk mendapatkan output dari model. Output ini berisi indeks-indeks ruangan yang direkomendasikan. Akhirnya, kode menampilkan hasil rekomendasi berbasis konten dengan menampilkan kolom-kolom relevan dari DataFrame (`df_cleaned`) menggunakan indeks-indeks yang diperoleh dari output model TensorFlow Lite.*

In [ ]:
# # Tampilkan hasil rekomendasi
# print("Content-Based Recommendations:")
# print(recommendations[['ID', 'Nama', 'Lantai', 'Fakultas', 'Kapasitas', 'Luas', 'Rating', 'Tujuan']])

## Finalizing

*Pada langkah terakhir ini, fokusnya adalah pada finalisasi model dan penggunaan praktisnya. Fungsi `load_model_and_recommend` dimaksudkan untuk memuat model dan menghasilkan rekomendasi berbasis konten untuk ruangan dengan ID tertentu. Matriks similarity kosinus dan DataFrame yang sudah dibersihkan dimuat kembali, kemudian fungsi `get_content_based_recommendations` dipanggil untuk menghasilkan rekomendasi. Jika ID ruangan tidak ditemukan, akan ditampilkan pesan kesalahan dan sebuah DataFrame kosong akan dikembalikan. Contoh penggunaan dilakukan dengan menggunakan ID ruangan 227, yang menghasilkan rekomendasi berdasarkan similarity tertinggi dengan ruangan tersebut. Hasil rekomendasi akhir ditampilkan dalam format yang sama dengan mencakup atribut-atribut seperti ID, Nama, Lantai, Fakultas, Kapasitas, Luas, Rating, dan Tujuan dari ruangan yang direkomendasikan.*

In [65]:
# Fungsi untuk memuat model dan membuat rekomendasi
def load_model_and_recommend(room_id, top_n=5):
    cosine_sim = np.load('cosine_sim.npy')
    df_cleaned = pd.read_csv('final_cleaned_data.csv')
    try:
        recommendations = get_content_based_recommendations(room_id, df_cleaned, cosine_sim, top_n)
        return recommendations
    except ValueError as e:
        print(e)
        return pd.DataFrame()

# Contoh penggunaan model yg telah disimpan
room_id =  514
final_recommendations = load_model_and_recommend(room_id)

# Tampilkan rekomendasi akhir
if not final_recommendations.empty:
    print("Final Content-Based Recommendations:")
    print(final_recommendations[['ID', 'Nama', 'Lantai', 'Fakultas', 'Kapasitas', 'Luas', 'Rating', 'Tujuan']])


Final Content-Based Recommendations:
     ID               Nama  Lantai             Fakultas  Kapasitas  Luas  \
84  514  Lab. Mikrobiologi       1  Fakultas Kedokteran         25  20.0   
36  534  OSCE CENTER 2- R1       2  Fakultas Kedokteran         12  16.0   
44  532  OSCE CENTER 2- R3       2  Fakultas Kedokteran         12  16.0   
46  531  OSCE CENTER 2- R4       2  Fakultas Kedokteran         12  16.0   
50  558   OSCE CENTER 1-R5       1  Fakultas Kedokteran         12  16.0   

    Rating   Tujuan  
84     3.3  Seminar  
36     3.8  Seminar  
44     3.0  Seminar  
46     4.1  Seminar  
50     3.6  Seminar  


## penerapan ke dalam tensorflow lite

In [ ]:
# # Load cleaned dataframe
# df_cleaned = pd.read_csv('final_cleaned_data.csv')

# # Pastikan file model tflite ada di jalur yang benar
# model_path = "content_based_model.tflite"

# # Periksa apakah file model ada
# if not os.path.exists(model_path):
#     raise ValueError(f"Model file '{model_path}' tidak ditemukan. Pastikan file tersebut ada di direktori yang benar.")

# # Load TensorFlow Lite model
# interpreter = tf.lite.Interpreter(model_path=model_path)
# interpreter.allocate_tensors()

# # Input details
# input_details = interpreter.get_input_details()
# output_details = interpreter.get_output_details()

# # Masukkan ID ruangan yang ingin direkomendasikan
# room_id = 227

# # Konversi ID ke bentuk yang sesuai dengan input model
# input_data = np.array([room_id], dtype=np.int32).reshape(input_details[0]['shape'])
# interpreter.set_tensor(input_details[0]['index'], input_data)

# # Jalankan inferensi
# interpreter.invoke()

# # Dapatkan output
# output_data = interpreter.get_tensor(output_details[0]['index'])

# # Tampilkan hasil rekomendasi
# recommended_indices = output_data[0]  # Ambil rekomendasi
# print("Content-Based Recommendations:")
# for idx in recommended_indices:
#     print(df_cleaned.iloc[idx][['ID', 'Nama', 'Lantai', 'Fakultas', 'Kapasitas', 'Luas', 'Rating', 'Tujuan']])
